In [ ]:
!git clone https://github.com/vdevictor96/fabric-federated-learning

Cloning into 'fabric-federated-learning'...
remote: Enumerating objects: 473, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 473 (delta 185), reused 293 (delta 110), pack-reused 105
Receiving objects: 100% (473/473), 128.79 MiB | 13.51 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Updating files: 100% (153/153), done.


In [ ]:
!ls

fabric-federated-learning  sample_data


In [ ]:
%cd fabric-federated-learning/federated-learning/client


/content/fabric-federated-learning/federated-learning/client


In [ ]:
%cd ..

/content/fabric-federated-learning/federated-learning


In [ ]:
git pull

SyntaxError: invalid syntax (<ipython-input-38-9c5a8574ec5a>, line 1)

# Bert Centralised Learning

Set the module directory to import python files (RUN JUST ONCE)

In [ ]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



FileNotFoundError: [Errno 2] No such file or directory: '/home/victor/_bcfl/fabric-federated-learning/federated-learning'

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# NVIDIA drivers not working
torch.set_default_device('cpu')
device = torch.device('cpu')

### Initialize variables

In [ ]:
from client.model.bert_tiny import get_bert_tiny_tokenizer

MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 2
TEST_BATCH_SIZE = 8
NUM_EPOCHS = 10
LEARNING_RATE = 1e-05
tokenizer = get_bert_tiny_tokenizer()
root = 'client/data/datasets/dep1_cleaned.csv'



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Loading Reddit Depression Dataloaders

In [ ]:
from client.data.reddit_dep import get_reddit_dep_dataloaders

print(device)
train_loader, eval_loader, test_loader = get_reddit_dep_dataloaders(root, tokenizer,
                                                                    train_size=0.8,
                                                                    eval_size=0,
                                                                    test_size=0.2,
                                                                    train_batch_size=TRAIN_BATCH_SIZE,
                                                                    eval_batch_size=EVAL_BATCH_SIZE,
                                                                    test_batch_size=TEST_BATCH_SIZE,
                                                                    max_len=MAX_LEN,
                                                                    device=device)


cuda


### Creating Bert Tiny Model

In [ ]:
from client.model.bert_tiny import get_bert_tiny_model
print(device)
bert_tiny = get_bert_tiny_model(device=device)
print(bert_tiny)
bert_tiny.to(device)


cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

In [ ]:
!pip install evaluate
!pip install accelerate


In [ ]:
!pip install transformers[torch]

In [ ]:
!pip show accelerate


Name: accelerate
Version: 0.25.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


### Loading Transformers Trainer

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from client.data.reddit_dep import get_reddit_dep_datasets

train_dataset, eval_dataset, test_dataset = get_reddit_dep_datasets(root, tokenizer, 0.2, 0.2, 0.6, MAX_LEN)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = evaluate.load("accuracy")
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="client/models/bert-tiny/", evaluation_strategy="epoch")

training_args = TrainingArguments(
    output_dir="client/models/bert-tiny/",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=bert_tiny,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

### Train model with Trainer

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.473232,0.791599
2,No log,0.471304,0.796446
3,No log,0.467820,0.799677
4,No log,0.466921,0.794830
5,No log,0.466692,0.794830


TrainOutput(global_step=100, training_loss=0.4651242446899414, metrics={'train_runtime': 14.1344, 'train_samples_per_second': 218.969, 'train_steps_per_second': 7.075, 'total_flos': 3932157388800.0, 'train_loss': 0.4651242446899414, 'epoch': 5.0})

### Train model

In [ ]:
from torch.optim import Adam
from transformers import get_scheduler


# Loss function
loss_function = torch.nn.CrossEntropyLoss()
# Optimizer and learning rate scheduler
optimizer = Adam(bert_tiny.parameters(), lr=LEARNING_RATE)


num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

NameError: name 'model' is not defined

### Test the trained model

In [ ]:
from client.test import test_text_class
# Test the model
test_text_class(bert_tiny, test_loader, device=device)

hola2
Accuracy of the network on the 619 test sentences: 75.76736672051696 %


75.76736672051696

In [ ]:
! pip install optuna
# ! pip install ray[tune]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.7 MB/s eta 0:00:00


RuntimeError: To use hyperparameter search, you need to pass your model through a model_init function.

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

def model_init():
  return AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny")    

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

def my_hp_space(trial):
    return {
         "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 15),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }

best_run = trainer.hyperparameter_search(hp_space=my_hp_space, n_trials=20, direction="maximize")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2024-01-10 13:11:39,742] A new study created in memory with name: no-name-980d5b0e-8cd2-482a-bac0-b565e8f65386
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.634978,0.739903
2,No log,0.632016,0.743134


[I 2024-01-10 13:11:51,410] Trial 0 finished with value: 0.7431340872374798 and parameters: {'learning_rate': 2.4094571379636772e-06, 'num_train_epochs': 2, 'seed': 1, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.7431340872374798.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.631083,0.746365
2,No log,0.593175,0.747981
3,No log,0.583324,0.747981
4,No log,0.580511,0.747981


[I 2024-01-10 13:12:14,492] Trial 1 finished with value: 0.7479806138933764 and parameters: {'learning_rate': 2.082084889808661e-05, 'num_train_epochs': 4, 'seed': 34, 'per_device_train_batch_size': 16}. Best is trial 1 with value: 0.7479806138933764.
